### National Geographic Dark Mode

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
from dash import Dash, html, dcc, Input, Output, State, ctx
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
import plotly.colors as pc
import plotly.io as pio

# --- BUG FIX: Force default template globally to avoid 'None' issues ---
pio.templates.default = "plotly_white"

# ======================================================
# CONSISTENT BIRD COLOR MAPPING
# ======================================================
BIRD_COLOR_MAP = {
    'Eric': "#E67E22",    # Purple
    'Nico': "#27AE60",    # Green  
    'Sanne': "#8E44AD"    # Blue
}

# Ensure project path is correct
sys.path.append(os.path.abspath(".."))

# ======================================================
# 1. DATA LOADING & PRE-PROCESSING
# ======================================================

# Load Data
df = pd.read_csv('../data/bird_migration.csv')

# --- PRE-SORT DATA ---
df['date_time'] = pd.to_datetime(df['date_time'])
df = df.sort_values(['bird_name', 'date_time'])

# --- ROBUST STOP DETECTION HELPER ---
def count_significant_stops(sub_df, speed_threshold=0.5, min_duration_hours=1):
    is_resting = sub_df['speed_2d'] < speed_threshold
    block_ids = (is_resting != is_resting.shift()).cumsum()
    block_durations = sub_df.groupby(block_ids)['date_time'].agg(lambda x: x.max() - x.min())
    block_is_rest = sub_df.groupby(block_ids)['speed_2d'].mean() < speed_threshold
    min_duration = pd.Timedelta(hours=min_duration_hours)
    return ((block_is_rest) & (block_durations > min_duration)).sum()

# ======================================================
# 2. DATA AGGREGATION
# ======================================================

# A. Altitude Stats
alt_stats = df.groupby('bird_name').agg(
    Max_Altitude=('altitude', 'max'),
    Avg_Altitude=('altitude', 'mean'),
    Min_Altitude=('altitude', 'min')
)

# B. Speed Stats (Flying only)
df_flying = df[df['speed_2d'] >= 0.5] 

speed_stats = df_flying.groupby('bird_name').agg(
    Max_Speed=('speed_2d', 'max'),
    Avg_Speed=('speed_2d', 'mean'), 
    Min_Speed=('speed_2d', 'min')
)

# C. Stopover Counts
stop_counts = df.groupby('bird_name').apply(
    count_significant_stops, 
    include_groups=False
)

# D. Merge All Stats
df_bird_stats = alt_stats.join(speed_stats).join(stop_counts.rename('Total_Rest')).reset_index()

# ======================================================
# 3. VISUALIZATION ENGINES (ALIGNED & CENTERED)
# ======================================================

# --- HELPER: ANIMATION LAYOUT (ALIGNED) ---
def get_animation_layout(title_text, y_label):
    return dict(
        uirevision=True, 
        
        # --- CENTERED TITLE ---
        title={
            'text': title_text, 
            'x': 0.5, 'xanchor': 'center',  # <--- Centered
            'font': {'size': 20, 'family': 'Arial, sans-serif', 'color': '#2C3E50'}
        },
        template='plotly_white', 
        paper_bgcolor='white',
        plot_bgcolor='white',
        
        margin=dict(t=60, b=50, l=40, r=40),
        xaxis=dict(title="", showticklabels=False, visible=True), 
        yaxis=dict(title=y_label, gridcolor='#e0e0e0'),
        
        # --- ALIGNED BUTTONS (Left Side) ---
        updatemenus=[dict(
            type="buttons",
            direction="left",
            showactive=False,
            x=0, y=-0.1,      # <--- Aligned with slider
            xanchor="left", yanchor="top",
            pad={"r": 10, "t": 0},
            bgcolor="white",
            buttons=[
                # Play
                dict(label="▶", method="animate", 
                     args=[None, dict(frame=dict(duration=100, redraw=True), fromcurrent=True)],
                     args2=[None, dict(frame=dict(duration=0, redraw=False), mode="immediate")]),
                # Pause 
                dict(label="⏸", method="animate", 
                     args=[[None], dict(frame=dict(duration=0, redraw=False), mode="immediate")])
            ]
        )],
        
        # --- ALIGNED SLIDER (Right Side) ---
        sliders=[dict(
            active=0,
            yanchor="top", y=-0.1,    # <--- Same height as buttons
            xanchor="left", x=0.15,   # <--- Starts after buttons
            len=0.85,                 # <--- Takes remaining width
            currentvalue=dict(font=dict(color="#2C3E50"), prefix="Date: ", visible=True, xanchor="right"),
            transition=dict(duration=100),
            pad=dict(b=10, t=0),
            bgcolor="#2f4158", activebgcolor="#E67E22", bordercolor="#2f4158", tickcolor="#2C3E50", font=dict(color="#2C3E50")
        )]
    )

# --- MAP ENGINE (STATIC) ---
def create_map(df):
    fig = go.Figure()
    if df.empty:
        fig.add_annotation(text="No data selected", x=0.5, y=0.5, showarrow=False)
        fig.update_layout(template="plotly_white", paper_bgcolor="white")
        return fig
    
    unique_birds = df['bird_name'].unique()
    for bird in unique_birds:
        dff = df[df['bird_name'] == bird]
        fig.add_trace(go.Scattergeo(
            lat=dff['latitude'], lon=dff['longitude'], mode='lines+markers', name=bird,
            line=dict(width=2, color=BIRD_COLOR_MAP.get(bird, 'gray')),
            marker=dict(size=6, opacity=0.8), hoverinfo='text', text=dff['bird_name']
        ))
    
    fig.update_geos(
        visible=True, resolution=50, showcountries=True, countrycolor="#A0A0A0",
        showcoastlines=True, coastlinecolor="#888888", showland=True, landcolor="#EAE7D6",      
        showocean=True, oceancolor="#D4F1F9", showlakes=True, lakecolor="#D4F1F9",     
        showrivers=True, rivercolor="#D4F1F9", projection_rotation=dict(lon=-10, lat=20),
        projection_type="orthographic", fitbounds="locations"
    )
    fig.update_layout(
        title={'text': "TRAJECTORY MAP", 'x': 0.5, 'xanchor': 'center', 'font': {'size': 20, 'family': 'Arial, sans-serif', 'color': '#2C3E50'}},
        margin={"r":0,"t":60,"l":0,"b":0}, paper_bgcolor="white", 
        legend=dict(yanchor="top", y=0.95, xanchor="left", x=0.05, bgcolor="rgba(255,255,255,0.8)"),
        template="plotly_white"
    )
    return fig

# --- BAR CHART ENGINE ---
def build_bar_chart(df_bird_stats, selected_bird, category, selected_stats):
    if not selected_bird or not selected_stats:
        return go.Figure().update_layout(title={'text': "Select options", 'x': 0.5}, paper_bgcolor='white', plot_bgcolor='white', height=500)

    cols_to_plot = []
    title_prefix, y_label = "", ""
    if category == "Altitude":
        y_label, title_prefix = "Altitude (m)", "ALTITUDE COMPARISON"
        for stat in selected_stats: cols_to_plot.append(f"{stat}_{category}")
    elif category == "Speed":
        y_label, title_prefix = "Speed (km/h)", "SPEED COMPARISON"
        for stat in selected_stats: cols_to_plot.append(f"{stat}_{category}")
    elif category == "Rest":
        y_label, title_prefix = "Count", "REST STOP FREQUENCY"
        cols_to_plot.append("Total_Rest")

    df_filtered = df_bird_stats[df_bird_stats['bird_name'].isin(selected_bird)].copy()
    valid_cols = [c for c in cols_to_plot if c in df_filtered.columns]
    if not valid_cols: return go.Figure().update_layout(title="No data", height=500, paper_bgcolor='white', plot_bgcolor='white')

    df_melted = df_filtered.melt(id_vars='bird_name', value_vars=valid_cols, var_name='Metric', value_name='Value')
    unique_birds = sorted(df_melted['bird_name'].unique())
    fig = go.Figure()
    
    for bird in unique_birds:
        bird_data = df_melted[df_melted['bird_name'] == bird].copy()
        base_color = BIRD_COLOR_MAP.get(bird, '#808080')
        fig.add_trace(go.Bar(
            x=bird_data['Metric'], y=bird_data['Value'], name=bird,
            marker=dict(color=base_color, line=dict(color='white', width=1)),
            text=bird_data['Value'].round(0).astype(int), texttemplate='%{text:,}', textposition='outside'
        ))

    fig.update_layout(
        title={'text': title_prefix, 'x': 0.5, 'xanchor': 'center', 'font': {'size': 20, 'family': 'Arial, sans-serif', 'color': '#2C3E50'}},
        template='plotly_white', legend=dict(orientation="h", y=1.1, x=1, xanchor="right"),
        margin=dict(t=60, b=60, l=60, r=200), yaxis=dict(title=y_label, gridcolor='#e0e0e0'),
        xaxis=dict(title="", showgrid=False), barmode='group', height=500, plot_bgcolor='white', paper_bgcolor='white'
    )
    return fig

# --- ALTITUDE LINE CHART ---
def build_line_chart_altitude(df, selected_bird):
    if not selected_bird: return go.Figure().update_layout(title="Select bird", paper_bgcolor='white')
    if isinstance(selected_bird, str): selected_bird = [selected_bird]
    
    df_filtered = df[df["bird_name"].isin(selected_bird)].copy().dropna(subset=["date_time", "altitude"]).sort_values(["bird_name", "date_time"])
    df_filtered["frame"] = pd.to_datetime(df_filtered["date_time"].dt.strftime("%Y-%m-%d"))
    df_daily = df_filtered.groupby(["bird_name", "frame"]).first().reset_index()
    df_daily["avg_val"] = df_daily.groupby("bird_name")["altitude"].expanding().mean().reset_index(level=0, drop=True)
    
    line_plot = px.line(df_daily, x='date_time', y='avg_val', color='bird_name', color_discrete_map=BIRD_COLOR_MAP, template={})
    scatter_plot = px.scatter(df_daily, x='date_time', y='avg_val', color='bird_name', color_discrete_map=BIRD_COLOR_MAP, animation_frame="frame", template={})

    for trace in line_plot.data: scatter_plot.add_trace(trace)
    
    layout = get_animation_layout("ALTITUDE OVER TIME", "Altitude (m)")
    layout['sliders'][0]['steps'] = [{'args': [[f['name']], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate'}], 'method': 'animate', 'label': pd.to_datetime(f['name']).strftime('%b %Y')} for f in scatter_plot.frames]
    
    scatter_plot.update_layout(**layout)
    return scatter_plot

# --- SPEED LINE CHART ---
def build_line_chart_speed(df, selected_bird):
    if not selected_bird: return go.Figure().update_layout(title="Select bird", paper_bgcolor='white')
    if isinstance(selected_bird, str): selected_bird = [selected_bird]
    
    df_filtered = df[df["bird_name"].isin(selected_bird)].copy().dropna(subset=["date_time", "speed_2d"]).sort_values(["bird_name", "date_time"])
    df_filtered["frame"] = pd.to_datetime(df_filtered["date_time"].dt.strftime("%Y-%m-%d"))
    df_daily = df_filtered.groupby(["bird_name", "frame"]).first().reset_index()
    df_daily["avg_val"] = df_daily.groupby("bird_name")["speed_2d"].expanding().mean().reset_index(level=0, drop=True)
    
    line_plot = px.line(df_daily, x='date_time', y='avg_val', color='bird_name', color_discrete_map=BIRD_COLOR_MAP, template={})
    scatter_plot = px.scatter(df_daily, x='date_time', y='avg_val', color='bird_name', color_discrete_map=BIRD_COLOR_MAP, animation_frame="frame", template={})

    for trace in line_plot.data: scatter_plot.add_trace(trace)
    
    layout = get_animation_layout("SPEED OVER TIME", "Speed (km/h)")
    layout['sliders'][0]['steps'] = [{'args': [[f['name']], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate'}], 'method': 'animate', 'label': pd.to_datetime(f['name']).strftime('%b %Y')} for f in scatter_plot.frames]
    
    scatter_plot.update_layout(**layout)
    return scatter_plot

# --- ANIMATED MAP (CUSTOM LAYOUT) ---
def build_animated_map(df, selected_bird):
    if not selected_bird: return create_map(pd.DataFrame())
    if isinstance(selected_bird, str): selected_bird = [selected_bird]
    
    df_filtered = df[df["bird_name"].isin(selected_bird)].copy().dropna(subset=["date_time", "latitude"]).sort_values(["bird_name", "date_time"])
    df_filtered["frame"] = pd.to_datetime(df_filtered["date_time"].dt.strftime("%Y-%m-%d"))
    df_daily = df_filtered.groupby(["bird_name", "frame"]).first().reset_index().sort_values("frame")
    
    # Path Logic
    df_hourly = df_filtered.groupby(["bird_name", "frame"]).first().reset_index()
    
    fig = px.line_geo(df_hourly, lat="latitude", lon="longitude", color="bird_name", color_discrete_map=BIRD_COLOR_MAP, line_group="bird_name", template={})
    fig_points = px.scatter_geo(df_daily, lat="latitude", lon="longitude", color="bird_name", color_discrete_map=BIRD_COLOR_MAP, size=np.array([10]*len(df_daily)), animation_frame="frame", animation_group="bird_name", template={})

    for trace in fig.data: fig_points.add_trace(trace)

    # --- CUSTOM ANIMATED MAP LAYOUT ---
    fig_points.update_layout(
        # 1. Centered Title
        title={
            'text': "ANIMATED MOVEMENT", 
            'x': 0.5, 'xanchor': 'center', 
            'font': {'size': 20, 'family': 'Arial, sans-serif', 'color': '#2C3E50'}
        },
        template='plotly_white',
        paper_bgcolor='white',
        plot_bgcolor='white',
        margin={"r":0,"t":50,"l":0,"b":50}, # Balanced margins to center map
        
        # 2. Geo Settings (Atlas Style)
        geo=dict(
            fitbounds="locations", 
            showcountries=True, countrycolor="#A0A0A0",
            showcoastlines=True, coastlinecolor="#888888",
            showland=True, landcolor="#EAE7D6",
            showocean=True, oceancolor="#D4F1F9",
            showlakes=True, lakecolor="#D4F1F9",
            showrivers=True, rivercolor="#D4F1F9",
            projection_type="natural earth"
        ),
        
        # 3. Aligned Buttons (Bottom Left)
        updatemenus=[dict(
            type="buttons",
            direction="left",
            showactive=False,
            x=0.05, y=0.05,           # Position inside the frame, bottom left
            xanchor="left", yanchor="bottom",
            pad={"r": 10, "t": 10},
            bgcolor="white",
            buttons=[
                # Green Play Button
                dict(
                    label="▶ Play",
                    method="animate",
                    args=[None, dict(frame=dict(duration=100, redraw=True), fromcurrent=True)],
                    args2=[None, dict(frame=dict(duration=0, redraw=False), mode="immediate")]
                ),
                # Orange Pause Button
                dict(
                    label="⏸ Pause",
                    method="animate",
                    args=[[None], dict(frame=dict(duration=0, redraw=False), mode="immediate")]
                )
            ]
        )],
        
        # 4. Aligned Slider (Next to Buttons)
        sliders=[dict(
            active=0,
            yanchor="bottom", y=0.05, # Align vertically with buttons
            xanchor="left", x=0.28,   # Start to the right of the buttons
            len=0.65,                 # Width of slider
            currentvalue=dict(font=dict(color="#2C3E50"), prefix="Date: ", visible=True, xanchor="right"),
            transition=dict(duration=100),
            pad=dict(b=10, t=50),
            
            # Slider Colors
            bgcolor="#2f4158",       # Dark Blue Track
            activebgcolor="#E67E22", # Orange Active Track
            bordercolor="#2f4158",
            tickcolor="#2C3E50",
            font=dict(color="#2C3E50"),
            
            # Steps logic
            steps=[{'args': [[f['name']], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate'}], 'method': 'animate', 'label': pd.to_datetime(f['name']).strftime('%b %Y')} for f in fig_points.frames]
        )]
    )
    
    # --- Manual Button Styling (Since updatemenus dict is limited) ---
    # We apply the specific colors by updating the menus directly after creation
    fig_points.update_layout(
        updatemenus=[
            dict(
                buttons=[
                    dict(args=[None, dict(frame=dict(duration=100, redraw=True), fromcurrent=True)], label="▶", method="animate"), # Play
                    dict(args=[[None], dict(frame=dict(duration=0, redraw=False), mode="immediate")], label="⏸", method="animate")   # Pause
                ],
                direction="left",
                pad={"r": 10, "t": 87},
                showactive=False,
                type="buttons",
                x=0.05,
                xanchor="left",
                y=0.05,
                yanchor="bottom",
                bgcolor="white",
                font=dict(color="#27AE60", size=16, weight="bold") # Green Text for Play
            )
        ]
    )
    
    return fig_points
# ======================================================
# 4. APP LAYOUT
# ======================================================

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
server = app.server

# --- INJECT CSS DIRECTLY INTO THE HEADER (Works on old Dash versions) ---
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>{%title%}</title>
        {%favicon%}
        {%css%}
        <style>
            /* Make Checkboxes Orange when checked */
            .form-check-input:checked {
                background-color: #E67E22 !important;
                border-color: #E67E22 !important;
            }
            /* Optional: Make radio buttons Orange too */
            .form-check-input[type=radio]:checked {
                background-image: url("data:image/svg+xml,%3csvg xmlns='http://www.w3.org/2000/svg' viewBox='-4 -4 8 8'%3e%3ccircle r='2' fill='%23fff'/%3e%3c/svg%3e");
            }
        </style>
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

# --- CUSTOM STYLES ---
DARK_CARD_STYLE = {
    "backgroundColor": "#2f4158", # Dark Blue
    "border": "none",
    "borderRadius": "10px",
    "boxShadow": "0 4px 8px rgba(0,0,0,0.3)",
    "color": "white" # Default text color for the card
}

WHITE_BUTTON_STYLE = {
    "backgroundColor": "white",    
    "color": "#2C3E50",            
    "border": "none",
    "fontWeight": "bold"
}

app.layout = dbc.Container([
    
    # --- 1. HERO BANNER SECTION ---
    dbc.Row([
        dbc.Col([
            html.Div([
                html.Img(
                    src=app.get_asset_url('background.gif'), 
                    style={'width': '100%', 'height': '200px', 'objectFit': 'cover', 'filter': 'blur(4px)', 'position': 'absolute', 'top': '0', 'left': '0', 'zIndex': '1'}
                ),
                html.Div(style={'position': 'absolute', 'top': '0', 'left': '0', 'width': '100%', 'height': '200px', 'backgroundColor': 'rgba(0, 0, 0, 0.4)', 'zIndex': '2'}),
                html.Div([
                    html.H1("Journeys In The Sky", className="display-3 fw-bold text-white"),
                    html.P("Global Bird Migration Tracker", className="lead text-light"),
                    html.P("Compare patterns, altitudes, and speeds across 3 seagulls - Eric, Nico and Sanne.", className="text-white fs-5 fw-bold")
                ], style={'position': 'relative', 'zIndex': '3', 'paddingTop': '30px', 'textAlign': 'center'})
            ], style={'position': 'relative', 'height': '200px', 'overflow': 'hidden', 'borderRadius': '0 0 15px 15px'})
        ], width=12)
    ], className="mb-4", style={"position": "sticky", "top": "0", "zIndex": "2000", "backgroundColor": "white"}),
    
    # --- MAIN CONTENT ROW ---
    dbc.Row([
        
        # --- LEFT SIDEBAR (Dark Blue) ---
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Global Filter", className="fw-bold text-white", style={"backgroundColor": "transparent", "borderBottom": "1px solid rgba(255,255,255,0.2)"}),
                dbc.CardBody([
                    html.Label("Select Birds", className="mb-2 fw-bold text-white"),
                    dbc.Checklist(
                        id='bird-name-filter',
                        options=[{'label': s, 'value': s} for s in sorted(df['bird_name'].unique())],
                        value=sorted(df['bird_name'].unique()),  
                        className="mb-3 text-white"
                    ),
                    dbc.Button("Select All Birds", id="btn-all-birds", size="sm", className="mt-1 w-100 shadow-sm", style=WHITE_BUTTON_STYLE),
                ])
            ], 
            className="mb-4 shadow-sm",
            # Combined Dark Style + Sticky Positioning
            style={**DARK_CARD_STYLE, "position": "sticky", "top": "220px", "zIndex": "1000", "height": "fit-content"})
        ], width=12, md=3), 
        
        # --- RIGHT COLUMN (Charts) ---
        dbc.Col([
            
            # Row 1: Static Map
            dbc.Row([
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([dcc.Graph(id='migration-map', style={'height': '75vh'}, config={'responsive': True})], style={'padding': '0'})
                    ], className="shadow-sm mb-5") 
                ], width=12)
            ]),
            
            html.Div(style={"height": "50px"}),

            # Row 2: Bar Chart + Chart Controls
            dbc.Row([
                # Bar Chart
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([dcc.Graph(id='main-bar-chart', style={'height': '50vh'}, config={'responsive': True})], style={'padding': '0'})
                    ], className="shadow-sm border-0 mb-5")
                ], width=12, md=9),

                # CHART CONTROLS (Dark Blue)
                dbc.Col([
                    dbc.Card([
                        dbc.CardHeader("Chart Controls", className="fw-bold text-white", style={"backgroundColor": "transparent", "borderBottom": "1px solid rgba(255,255,255,0.2)"}),
                        dbc.CardBody([
                            html.Label("Choose Data Category", className="fw-bold text-white"),
                            dbc.RadioItems(
                                id='category-selector',
                                options=[{'label': ' Altitude', 'value': 'Altitude'}, {'label': ' Speed', 'value': 'Speed'}, {'label': ' Rest Stops', 'value': 'Rest'}],
                                value='Altitude', 
                                className="mb-3 text-white", 
                                inputClassName="me-2"
                            ),
                            html.Hr(style={'borderColor': 'rgba(255,255,255,0.2)'}),
                            html.Div([
                                html.Label("Select Statistics", className="fw-bold text-white"),
                                dbc.Checklist(
                                    id='statistic-checklist',
                                    options=[{'label': ' Maximum', 'value': 'Max'}, {'label': ' Average', 'value': 'Avg'}, {'label': ' Minimum', 'value': 'Min'}],
                                    value=['Max', 'Avg', 'Min'],  
                                    className="mb-3 text-white"
                                ),
                            ], id='stats-container') 
                        ])
                    ], className="shadow-sm sticky-top", 
                       style={**DARK_CARD_STYLE, "top": "220px"}) # Applied Dark Style here too
                ], width=12, md=3)
            ]),

            html.Div(style={"height": "200px"}),

            # Row 3: Animated Map + Line Charts
            dbc.Row([
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([dcc.Graph(id="animated-map", style={"height": "75vh"}, config={'responsive': True})], style={"padding": "0"})
                    ], className="shadow-sm")
                ], width=12, md=6),

                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([dcc.Graph(id="line-chart-altitude", style={"height": "35vh"}, config={'responsive': True})], style={"padding": "0"})
                    ], className="shadow-sm border-0 mb-4"), 
                    
                    dbc.Card([
                        dbc.CardBody([dcc.Graph(id="line-chart-speed", style={"height": "35vh"}, config={'responsive': True})], style={"padding": "0"})
                    ], className="shadow-sm border-0")
                ], width=12, md=6),
            ])
        ], width=12, md=9), 
    ]), 
], fluid=True, style={'backgroundColor': "#EAF4F9", 'minHeight': '100vh'})

# ======================================================
# 5. CALLBACKS
# ======================================================

@app.callback(Output('migration-map', 'figure'), Input('bird-name-filter', 'value'))
def update_map(selected_bird_names):
    if not selected_bird_names: return create_map(pd.DataFrame())
    filtered = df[df['bird_name'].isin(selected_bird_names)]
    return create_map(filtered)

@app.callback([Output('statistic-checklist', 'options'), Output('statistic-checklist', 'value'), Output('stats-container', 'style')], Input('category-selector', 'value'), State('statistic-checklist', 'value'))
def update_stat_options(category, current_values):
    if category == 'Rest': return [{'label': ' Total Count', 'value': 'Total'}], ['Total'], {'display': 'none'}
    elif category == 'Speed':
        valid_values = [v for v in current_values if v in ['Max', 'Avg']]
        return [{'label': ' Maximum Speed', 'value': 'Max'}, {'label': ' Average Speed', 'value': 'Avg'}], valid_values if valid_values else ['Max', 'Avg'], {'display': 'block'}
    else: 
        valid_values = [v for v in current_values if v in ['Max', 'Avg', 'Min']]
        return [{'label': ' Maximum', 'value': 'Max'}, {'label': ' Average', 'value': 'Avg'}, {'label': ' Minimum', 'value': 'Min'}], valid_values if valid_values else ['Max', 'Avg', 'Min'], {'display': 'block'}

@app.callback(Output('main-bar-chart', 'figure'), [Input('bird-name-filter', 'value'), Input('category-selector', 'value'), Input('statistic-checklist', 'value')])
def update_chart(selected_bird, category, selected_stats):
    return build_bar_chart(df_bird_stats, selected_bird, category, selected_stats)

@app.callback(Output('line-chart-altitude', 'figure'), Input('bird-name-filter', 'value'))
def update_line_chart_altitude(selected_bird):
    return build_line_chart_altitude(df=df, selected_bird=selected_bird)

@app.callback(Output('line-chart-speed', 'figure'), Input('bird-name-filter', 'value'))
def update_line_chart_speed(selected_bird):
    return build_line_chart_speed(df=df, selected_bird=selected_bird)

@app.callback(Output('animated-map', 'figure'), Input('bird-name-filter', 'value'))
def update_animated_map(selected_bird):
    return build_animated_map(df=df, selected_bird=selected_bird)

@app.callback(Output('bird-name-filter', 'value'), Input('btn-all-birds', 'n_clicks'), State('bird-name-filter', 'options'), prevent_initial_call=True)
def select_all_species(n_clicks, options):
    return [opt['value'] for opt in options]

# --- RUN ---
if __name__ == '__main__':
    app.run(debug=True, port=8058)